## Cargamos las librerias

In [ ]:
import pandas as pd
import numpy as np

## Cargamos las data

In [ ]:
data_review=pd.read_csv('reviews.csv')
data_items=pd.read_csv('items.csv')
data_games=pd.read_csv('games.csv')

# data_review EDA y ETL

Observamos el numero de filas del dataFrame data_review

In [ ]:
data_review.isnull().sum()

Vamos a eliminar las siguientes columnas debido a la gran cantidad de valores vacíos que contienen.

In [ ]:
data_review.drop(columns=['funny','last_edited'], inplace=True)

### Se procederá a eliminar las siguientes columnas por las razones especificadas:

- `helpful`: No aporta información relevante significativa.
- `recommend`: Esta columna será convertida a dummy con las categorías `recommend_False` y `recommend_True`.
- `review` y `review_normalizado`: Estas columnas se eliminarán ya que están reemplazadas por la columna `sentiment_analysis`.


In [ ]:
data_review.drop(columns=['helpful','recommend','review','review_normalizado'], inplace=True)

Ahora nos disponemos a corregir el formato de fecha en la columna `posted`. El primer paso será eliminar la palabra 'posted' de dicha columna.

data_review['posted']=data_review['posted'].str.replace('Posted','')

Vamos a eliminar los puntos al final de cada una de nuestras fechas.

In [ ]:
data_review['posted']=data_review['posted'].str.replace('.','')

Dado que en nuestra columna `posted` tenemos fechas que no incluyen el año, procederemos a dividir esta columna en dos utilizando la coma como punto de división. Dado que el año se encuentra antes de la coma, obtendremos dos nuevas columnas: una llamada `year` para el año y otra `month` para el mes.

In [ ]:
data_review[['mont', 'year']]=data_review['posted'].str.split(',', expand=True)

Vamos a eliminar el punto que se encuentra en nuestra columna `year` (año).

In [ ]:
data_review['year']=data_review['year'].str.replace('.','')

Estamos filtrando todos los valores en los que nuestra columna `year` no es nula.

In [ ]:
data_review=data_review[data_review['year'].notna()]

## data_games **EDA y ETL**

Vamos a observar las filas vacías de nuestro DataFrame `data_games`.

In [ ]:
data_games.isnull().sum()

En el DataFrame `data_games`, eliminaremos las filas que contengan más de 5 valores faltantes.

In [ ]:
data_games.dropna(inplace=True, thresh=5)

Vamos a filtrar aquellas filas donde las columnas 'genres' y 'tags' están vacías, con el fin de descartar aquellos videojuegos que no tengan género especificado.

In [ ]:
data_games = data_games[(data_games['genres'].notna()) & (data_games['tags'].notna())]

Dado que nuestra columna `genres` está compuesta por filas llenas de listas que tienen formato tipo string, debemos convertir esta columna de tipo string a tipo lista.

In [ ]:
data_games['genres'] = data_games['genres'].apply(ast.literal_eval)

Vamos a obtener el primer elemento de las listas que se encuentran dentro de la columna `genres` y lo guardaremos en una nueva columna llamada `primer_genero`.

In [ ]:
data_games['primer_genero'] = data_games['genres'].str.get(0)

Al observar una lista de elementos únicos en la columna `genres`, notamos que existen géneros que no pertenecen a videojuegos.

In [ ]:
data_games['primer_genero'].unique()

array(['Action', 'Free to Play', 'Casual', 'Adventure', 'Racing',
       'Strategy', 'Indie', 'RPG', 'Simulation', 'Sports',
       'Massively Multiplayer', 'Animation &amp; Modeling', 'Utilities',
       'Education', 'Design &amp; Illustration', 'Audio Production',
       'Video Production', 'Software Training', 'Accounting',
       'Web Publishing', 'Photo Editing', 'Early Access'], dtype=object) 

En base al anterior paso, elaboramos una lista de los géneros que no pertenecen a la categoría de videojuego con el fin de descartarlos.

In [ ]:
genero_descartar= ['Animation &amp; Modeling', 'Utilities',
       'Education', 'Design &amp; Illustration', 'Audio Production',
       'Video Production', 'Software Training', 'Accounting',
       'Web Publishing', 'Photo Editing', 'Early Access']

Filtramos aquellos juegos que no pertenecen a la lista `genero_descartar`.

In [ ]:
data_games['genres_games']=data_games[~data_games['primer_genero'].isin(genero_descartar)]['primer_genero'] # se usa el caracter ~ para negar la condicion

De nuestro DataFrame `data_games`, vamos a crear un DataFrame auxiliar que contenga solo las columnas `id` y `genres_games` con el propósito de realizar un inner join con el DataFrame `data_items`.

In [ ]:
df_auxiliar= data_games[['genres_games', 'id']]

Cambiamos el nombre de la columna `id` a `item_id` para que coincida con el nombre de la columna que tenemos en el archivo `data_items` y así poder hacer el inner join.

In [ ]:
## cambiamos el nombre de la columna id a item_id para que concida con la columna que tenemos en el archivo data_items
df_auxiliar['item_id']=df_auxiliar['id']

## eiminamos la columna id
df_auxiliar=df_auxiliar.drop(columns='id')

Unimos los dos DataFrames a través de la columna `item_id`.

In [ ]:
data_items=pd.merge(data_items, df_auxiliar, on='item_id', how='inner')

Filtramos los valores donde la columna `genres_games` es diferente a vacío.

In [ ]:
data_items=data_items[data_items['genres_games'].notna()]

## **Nota**

El DataFrame original de `data_items` tenía originalmente 5153209 filas. Después de aplicar el filtro, esto se reduce a 4188785 filas. Esto se debe a que eliminamos categorías que no pertenecían a videojuegos, lo que resultó en la eliminación de `item_id` que no correspondían a ninguna categoría de videojuego, reduciendo así el número de filas.

In [ ]:
data_items.shape

Guardamos los cambios en los DataFrames para asegurarnos de que queden registradas las modificaciones realizadas.

In [ ]:
data_items.to_csv('items.csv')
data_games.to_csv('games.csv')